In [ ]:
pip install requests boto3 pandas sqlalchemy psycopg2


In [ ]:
import zipfile
import os

# Configuration
local_zip_file = r"ZIP FILE LOCTION"  # Path to the downloaded ZIP file
extract_folder = r"EXTRACTED FOLDER PATH"     # Folder to extract files into

# Create extraction folder if it doesn't exist
if not os.path.exists(extract_folder):
    os.makedirs(extract_folder)

# Unzip the downloaded file
def unzip_file(zip_file_path, extract_to):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Extracted {zip_file_path} to {extract_to}")

# Extract the ZIP file
unzip_file(local_zip_file, extract_folder)


In [ ]:
import boto3

# Configure your S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id='Acess key ID',
    aws_secret_access_key='Secret Access key',
    region_name='us-east-1'
)

# Local file path and S3 bucket details
local_file_path = r"Path to file"
bucket_name = 'Bucket_name'
s3_file_key = 'Path to file'  # Change 'folder-name' as needed

# Upload the file
try:
    s3.upload_file(local_file_path, bucket_name, s3_file_key)
    print(f"File uploaded successfully to {bucket_name}/{s3_file_key}")
except Exception as e:
    print(f"Error uploading file: {e}")



In [ ]:
!pip install SQLAlchemy pymysql

In [ ]:
from sqlalchemy import create_engine

# Database credentials and connection information
db_user = 'admin'       # RDS database username
db_password = 'PASSWORD'   # RDS database password
db_host = 'Endpoint link'    # RDS endpoint
db_port = '3306'                # RDS port, default is 3306 for MySQL
db_name = 'Database name'  # Name of your RDS database

# Create the connection string for the RDS instance
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Test the connection
try:
    with engine.connect() as connection:
        # If connection is successful, print message
        print("Connection to the database was successful!")
except Exception as e:
    print("Error connecting to the database:", e)


In [ ]:
import boto3
import pymysql
import json

# AWS and DB configurations
s3_bucket = 'Bucket_Name'
s3_key = 'File to path'
db_host = 'Endpoint link'
db_user = 'admin'
db_password = 'PASSWORD'
db_name = 'TEST'

# Connect to S3
s3 = boto3.client('s3')
response = s3.get_object(Bucket=s3_bucket, Key=s3_key)
json_content = response['Body'].read().decode('utf-8')

# Parse JSON content
data = json.loads(json_content)

# Connect to RDS
connection = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)
cursor = connection.cursor()

# Insert JSON data into the table
sql = "INSERT INTO json_data (data) VALUES (%s)"
for record in data:
    cursor.execute(sql, (json.dumps(record),))

# Commit and close connection
connection.commit()
cursor.close()
connection.close()

print("JSON data uploaded successfully!")


In [ ]:
import boto3
import pymysql
import json

# AWS and DB configurations
s3_bucket = 'Buckey_name'
s3_key = 'Path to file'
db_host = 'Endpoint link'
db_user = 'admin'
db_password = 'PASSWORD'
db_name = 'DATABASE NAME'

# Connect to S3
s3 = boto3.client('s3')
response = s3.get_object(Bucket=s3_bucket, Key=s3_key)
json_content = response['Body'].read().decode('utf-8')

# Parse JSON content
data = json.loads(json_content)

# Connect to RDS
connection = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)
cursor = connection.cursor()


# Extract and insert data
cik = data['cik']
entity_name = data['entityName']
shares = data['facts']['dei']['EntityCommonStockSharesOutstanding']['units']['shares']

for entry in shares:
    end_date = entry['end']
    value = entry['val']
    sql = "INSERT INTO extracted_data (cik, entityName, end_date, value) VALUES (%s, %s, %s, %s)"
    cursor.execute(sql, (cik, entity_name, end_date, value))

# Commit changes and close connection
connection.commit()
cursor.close()
connection.close()

print("Data inserted successfully!")